In [ ]:
import pandas as pd
from datetime import datetime
import pywhatkit as wpp
import dataframe_image as dfi
import pyautogui

Funciones

In [ ]:
def dias():
    reference_day = datetime(2024, 1, 8)
    today = datetime.now()
    deltaday = (today - reference_day).days

    sem = int(deltaday // 7 % 4)
    mes = today.month - reference_day.month +1
    year = 2024
    return sem, mes, year

In [ ]:
def clean_asig(df):
    # Solo Colunnas necesarias. Filtro solo Cadereyta
    df = df.rename(columns={'MATERIA':'GRUPO','MATERIA.1':'MATERIA'})
    df = df[['MATRICULA','NOMBRE', 'CE', 'MATERIA']]
    
    # Matriculas en MAYUS
    df['MATRICULA'] = df['MATRICULA'].str.upper()
    
    cade = ('CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA')
    df = df[df['CE'].str.contains('|'.join(cade), case = False, na = False)]
    return df

In [ ]:
def colab():
    print('Grupos Colaborativos')
    
    gcol = pd.read_csv('Infodata/LinksGColab.csv', index_col=0)
    
    display(gcol.iloc[:,[0,1]])
    
    grupo = str(input('Grupo ?'))
    link = gcol[gcol.iloc[:,0] == grupo].iloc[:,2].values[0]
    grupo = gcol[gcol.iloc[:,0] == grupo].iloc[:,1].values[0]
    try: 
        asig = pd.read_csv('Data/' + "Asignacion.csv", index_col= 0)
    except FileNotFoundError or FileExistsError:
        mes = datetime.now().month
        asig = pd.read_excel('Data/Main/' + "Román - M{} Seguimiento.xlsx".format(mes), sheet_name = "Asignación")
        clean_asig(asig).to_csv('Data/' + 'Asignacion.csv')
        print('Asignacion csv Created')
        asig = pd.read_csv('Data/' + "Asignacion.csv", index_col= 0)
    if grupo:
        asig = asig[asig['MATERIA'] == grupo]
    asig_merged = pd.merge(asig, database[['MATRICULA', 'Teléfono', 'Teléfono2']], on = 'MATRICULA')
    return asig_merged, link

In [ ]:
def test():
    print('Enviar Test de Aprendizaje')
    path = 'Data/'
    df = pd.read_csv(path + 'PendientesTest.csv')
    df_merge = pd.merge(df, database[['MATRICULA', 'Teléfono', 'Teléfono2']], on = 'MATRICULA')
    return df_merge

In [ ]:
def result_test():
    print('Resultados Test de Aprendizaje')
    path = 'Data/'
    df = pd.read_csv(path + 'TestAprendizaje.csv')
    df_merge = pd.merge(df, database[['MATRICULA', 'Teléfono', 'Teléfono2']], on = 'MATRICULA')
    return df_merge

In [ ]:
def noinicios():
    print('No Inicios')
    path = 'Data/'
    main = 'Main/'
    try:
        df = pd.read_csv(path + 'NoInicios.csv')
    except FileNotFoundError or FileExistsError:
        sem , mes, year = dias()
        df = pd.read_excel(path + main +"BASE BLENDED NL S{:d}M{:02d} {}.xlsx".format(sem, mes, year), sheet_name = 'NO INICIO')
        df = df[['TIPO DE BAJA', 'NOMBRE EN LA BASE', 'MATRICULA EDCON','CE CANALIZADO','Fecha','SERVICIO','CARRERA','M1','Fecha de inicio','Correo','Teléfono','Teléfono 2','Asesor Financiero','Asesor Escolar',]]
        df = df.rename(columns={'NOMBRE EN LA BASE':'NOMBRE','MATRICULA EDCON':'MATRICULA', 'Teléfono 2':'Teléfono2'})
        df = df[df['CE CANALIZADO'].isin(['CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA'])]
        df.to_csv(path + 'NoInicios.csv')
    return df

In [ ]:
def result_induccion():
    df = pd.read_csv('infodata/' + 'Inducción_final.csv')
    df = df[['NOMBRE', 'MATRICULA', 'CE CANALIZADO', 'Availability', 'Calificación Final','Proyecto Modular', 'Proyecto Integrador', 'MODELO EDUCATIVO']]
    df = df[df['CE CANALIZADO'] == 'CE CADEREYTA']
    df = pd.merge(df, database[['MATRICULA', 'CARRERA','Teléfono', 'Teléfono2']], on = 'MATRICULA')
    return df 

In [ ]:
def save_infodata(df,long):
    df2 = pd.DataFrame(df, columns=['Matricula']) 
    if df2.empty:    
        df2.loc[-1] = datetime.now().strftime('%d/%m/%Y')
        df2.index = df2.index+1
        df2 = df2.sort_index()
    else:
        df2.loc[long] = datetime.now().strftime('%d/%m/%Y')
        df2.iloc[long+1:] = df2.iloc[long:-1].values

    df2.to_csv('Infodata/' + 'complementos_contactados.csv', index=False)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
def print_image(imagen):
    # Lee la imagen
    figure = Image.open(imagen)
    # Muestra la imagen
    plt.figure(figsize=(2,2))
    plt.imshow(figure)
    plt.axis('off')  # Desactiva los ejes
    plt.show()

In [ ]:
def promos(df):
    hoy = pd.to_datetime(datetime.now())
    df['Fecha de inicio'] = pd.to_datetime(df['Fecha de inicio'])
    def calcular_meses(fecha):
        delta = hoy - fecha
        return delta.days // 30  # Aproximar a meses

    df['MesesPasados'] = df['Fecha de inicio'].apply(calcular_meses)
    return df

Codigo

In [ ]:
path = 'Data/'
main = 'Main/'

try:
    database = pd.read_csv(path + "BASE CNCI.csv") 
except FileNotFoundError or FileExistsError: 
    %run DatabaseCNCI.ipynb
    print('Database CNCI.csv Created')
    database = pd.read_csv(path + "BASE CNCI.csv")

In [ ]:
path2 = 'Infodata/'
# imagen = 'img/Cerro.jpeg'

#Inicializar la infodata
matric_error = pd.read_csv('Infodata/matriculas_error.csv')
matric_error = matric_error.dropna().values.flatten().tolist()
try:    
    matric_yacontactadas = pd.read_csv('Infodata/matriculas_yacontactadas.csv') 
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist()
except FileNotFoundError:
    matric_yacontactadas = list()
long = len(matric_yacontactadas) 

Opciones

In [ ]:
option = 0
while option not in (1, 2, 3, 4):
    option = int(input('1.- Colab \n2.- Test Aprendizaje \n3.- No inicios  \n4.- Resultados Induccion'))
    if option == 1: 
        df, link = colab()
    elif option == 2:
        option2 = int(input('1.- Envio \n2.- Resultados '))
        if option2 == 1:
            link = 'https://forms.office.com/r/xjAa2rPUhf'
            df = test()
        elif option2 == 2:
            df = result_test()
    elif option == 3:
        df = noinicios()
        df = promos(df)
    elif option == 4:
        df = result_induccion()
        imagendf = input('Imprimir imagenes: 1')

Ciclo de mensajes

In [ ]:
for index, row in df.iterrows():
    alumno = row['NOMBRE']
    matricula = row['MATRICULA']
    telefono = row['Teléfono']
    
    mensaje = 'Hola, que tal {} {}, soy _Román Rodríguez_ tu Tutor Presencial del CNCI'.format(alumno, matricula)
    if option == 1:
        materia = row['MATERIA']
        mensaje = '{}, me comunico contigo para invitarte al grupo colaborativo de la materia: *{}* con la finalidad de que podamos entablar conversación por ahí sobre los Temas Relacionados al Curso y además tener apoyo entre los compañeros. \n\nSi gustas a continuación te dejo el enlace: {}'.format(mensaje, materia, link)
    elif option == 2:
        if option2 == 1:
            mensaje = '{}, me comunico contigo para invitarte a contestar el *Test de Aprendizaje*, el cual nos ayuda a saber cuál es la mejor manera en la que cada uno se le facilita aprender.\n Una vez contestado te agradecería que me mandes captura de que ya lo contestaste. \n\n Te dejo el enlace: {}'.format(mensaje, link)
        elif option2 == 2:
            resultado = row['Estilo de Aprendizaje'].title() + ' ' + row['Resultado'].title()
            mensaje = '{} te saludo de vuelta _Román Rodríguez_ Tutor Presencial del CNCI, me comunico contigo para enviarte los resultados del _Test de Aprendizaje_ que contestaste, con el cual obtuvimos el *mejor método de aprendizaje* para tí. Tu _Estilo de Aprendizaje_ es el siguiente: *{}*'.format(mensaje, resultado)
            imagen = 'img/ResultadosTestAprendizaje/'+ resultado+'.png'
    elif option == 3:
        fechaIn = row['Fecha de inicio'].strftime('%d/%m/%Y')
        mesesIn = row['MesesPasados']
        imagen = 'Img/Promo12menos.jpeg' if mesesIn < 12 else 'Img/Promo12mas.jpeg'
        mensaje = '*REMOTA TU CURSO DE INDUCCIÓN CON ESTA SÚPER PROMO* \nBuen día, {}. Me presento soy _Román Rodríguez_ Tutor Presencial del CNCI, me comunico contigo para invitarte a _RETOMAR_ tu curso de *Inducción*, así dar un nuevo inicio a _Tu Carrera Profesional_!!! Tu primer intento fue el día {}, dandonos un total de {} meses hasta la fecha de hoy, aún es buen momento para retomar. Mencioname si estas interesad@ y con gusto te contacto, ten un buen día! \n Próximo Inicio: *Lunes 10 de Junio*!!!'.format(alumno, fechaIn, mesesIn)
    elif option == 4:
        status = row['Availability']
        carrera = row['CARRERA']
        imagen = 'img/Induccion/Induccion_'+matricula+'.png'
        if  imagendf == 1:
            calif = df[df['MATRICULA'] == matricula][['NOMBRE', 'MATRICULA','Proyecto Modular', 'Proyecto Integrador', 'Calificación Final']]
            dfi.export(calif, imagen)
        if status == 'Acreditado':
            mensaje = '*Felicidades!!!!* {} {}. \nTe mando este mensaje para hacerte saber que _Acreditaste_ tu curso de *INDUCCIÓN*, por lo que el siguiente mes iniciaras tu carga de materias de {}. \nTe adjunto las calificaciones de este curso: \n\nPS: Calificación Final es en base a 10.'.format(alumno, matricula,carrera)
        else:
            mensaje = 'Hola, que tal {} {}, te hablo para notificarte sobre tu calificaciones del curso de *Inducción*, al parecer NO fuiste Acreditado, por lo que no podrás acceder a tu carga de materias. \nSin embargo puedes reprogramar tu Inducción, el costo lo verificamos con tu *AF*, si estás interesado, hazmelo saber para comunicarme con él. \nAdemás el costo de tu primer colegiatura se respetará hasta que acredites la Inducción. Por lo que no lo perderás. \nTe adjunto las calificaciones de este curso: \n\nPS: Calificación Final es en base a 10.'.format(alumno, matricula)

    if not pd.isna(telefono):
        telefono = '+52 ' + str(telefono)
    if matricula in matric_error:
        if not pd.isna(row['Teléfono2']):
            telefono2 = '+52 ' + str(int(row['Teléfono2']))
            telefono = telefono2
            print('Matricula con error: {}, Alumno: {}'.format(matricula, alumno))
    if matricula in matric_yacontactadas:
        print('Matricula antes contactada: {}, Alumno: {} '.format(matricula, alumno))    
    else:
        print('Enviado a: {} {} '.format(matricula, alumno))
        # print_image(imagen)
        print('--------------------')
        matric_yacontactadas.append(matricula)
        wpp.sendwhats_image(telefono, imagen, mensaje, 15, True)
        
save_infodata(matric_yacontactadas, long)